# SciPy

- [SciPy API reference](https://docs.scipy.org/doc/scipy/reference/index.html)
- [ndimage](https://docs.scipy.org/doc/scipy/reference/ndimage.html)

## SciPy: optimization

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def rosenbrock_f(a, b):
    """Return the Rosenbrock function, Jacobian & Hessian.

    Parameters
    ----------
    a, b : float
        Parameters defining the surface.  Typical values are a=1, b=100.

    Notes
    -----
    The Rosenbrock function has a minimum of 0 at ``(a, a**2)``.

    """
    def f(x, y):
        return (a - x)**2 + b * (y - x**2) ** 2

    def J(x, y):
        return np.array([-2 * (a - x) - 4 * b * x * (y - x**2),
                         2 * b * (y - x ** 2)])

    def H(x, y):
        return np.array([[2, -4 * b * x],
                         [-4 * b * x, 2 * b]])

    return f, J, H

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
rosenbrock, rosenbrock_J, rosenbrock_H = rosenbrock_f(a=1, b=100)

def plot_rozen():
    # Our first 3D plot!
    fig = plt.figure(figsize=(6, 5))
    ax = Axes3D(fig, azim=-128, elev=43)

    x = np.linspace(-2, 2)
    y = np.linspace(-1.25, 3)
    xg, yg = np.meshgrid(x, y)

    surf = ax.plot_surface(xg, yg, rosenbrock(xg, yg), rstride=1, cstride=1,
                           linewidth=0, antialiased=False, cmap='viridis', norm=LogNorm())

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('f(x, y)')
    ax.set_title('Rosenbrock landscape')
    
    return ax
    
plot_rozen();

In [ ]:
from scipy import optimize

x0 = (-0.5, 2.5)

optimize.minimize(lambda p: rosenbrock(*p), x0=x0, method='BFGS')

In [ ]:
path = [x0]

optimize.minimize(lambda p: rosenbrock(*p), x0=x0, method='BFGS',
                  callback=lambda p: path.append(p))

path = np.array(path)
z = rosenbrock(*path.T)  # equivalent to `rosenbrock(path[:, 0], path[:, 1])`
xyz = np.hstack([path, z[:, np.newaxis]])

ax = plot_rozen()
ax.plot(*xyz.T);

In [ ]:
methods = ['Nelder-Mead',
           'Powell',
           'CG',
           'BFGS',
           'Newton-CG',
           'L-BFGS-B',
           'TNC',
#           'COBYLA',   # does not support callbacks
           'SLSQP',
           'trust-ncg']

In [ ]:
def optimization_paths():
    rosenbrock, rosenbrock_J, rosenbrock_H = rosenbrock_f(a=1, b=100)
    path = {}

    fig, axes = plt.subplots(4, 3, figsize=(10, 10))
    fig.tight_layout(h_pad=2)
    fig.subplots_adjust(top=0.9)
    fig.delaxes(axes[0, 0])
    fig.delaxes(axes[0, 2])

    x, y = np.ogrid[-2:2:0.05, -1:3:0.05]
    extent = (-2, 2, -1, 3)

    z = rosenbrock(x, y).T
    axes[0, 1].matshow(z + 1e-3, norm=LogNorm(), origin='lower', extent=extent)

    x0 = (-0.5, 2.5)

    for n, method in enumerate(methods):
        print('Optimizing with {}'.format(method))

        path = [x0]
        res = optimize.minimize(lambda p: rosenbrock(*p),
                                x0=x0,
                                jac=lambda p: rosenbrock_J(*p),
                                hess=lambda p: rosenbrock_H(*p),
                                method=method,
                                callback=lambda p: path.append(p))

        path = np.array(path)
        px, py = path.T

        ax = axes.flat[n + 3]

        ax.contour(z, extent=extent, norm=LogNorm(), alpha=0.5)
        ax.plot(px, py, linewidth=3, color='black')
        ax.set_aspect('equal')
        ax.scatter(path[-1, 0], path[-1, 1])
        ax.set_title(method)

    ax.legend()
    
optimization_paths()

### Exercise

1. Read the docstring for `optimize.minimize` (i.e., type `optimize.minimize?` into the notebook and execute)
2.  As above, plot the optimization path in 3D for the Six Hump Camel function:

<img src="sixhumpcamel.png"/>